# Checking PyPSA and Gurobi installation

This is a very simple script to test that PyPSA and Gurobi installations are working properly.

It is based on the example "Unit commitment" available as a Jupyter notebook at https://pypsa.readthedocs.io/en/latest/examples/unit-commitment.html



In [11]:
import pypsa

Initiallize the network, set 4 time steps in the snapshots.

In [12]:
network = pypsa.Network()

network.set_snapshots(range(4))

Add one bus named "bus" to the network and add two generators (coal and gas).

Include the data regarding nominal power (p_nom), marginal cost (marginal_cost), and minimum power (p_min_pu) that needs to be dispatched by every generator. To enable unit commitment on a generator, set its attribute committable = True.

Add the load to the bus.

In [14]:
network.add("Bus","bus")


network.add("Generator",
            "coal",
            bus="bus",
            committable=True,
            p_min_pu=0.3,
            marginal_cost=20,
            p_nom=10000)

network.add("Generator",
            "gas",
            bus="bus",
            committable=True,
            marginal_cost=70,
            p_min_pu=0.1,
            p_nom=1000)

network.add("Load",
            "load",
            bus="bus",
            p_set=[4000,6000,5000,800])

AssertionError: Failed to add Bus component bus because there is already an object with this name in buses

Find the optimal solution using Gurobi as solver.

In this case, we are optimising the dispatch of every generator to minimise the total system cost.


In [15]:
network.optimize(solver_name='gurobi')

INFO:linopy.model: Solve problem using Gurobi solver


Read LP format model from file /tmp/linopy-problem-rp3pzlok.lp
Reading time = 0.01 seconds
obj: 36 rows, 32 columns, 84 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 1 logical processors, using up to 1 threads

Optimize a model with 36 rows, 32 columns and 84 nonzeros
Model fingerprint: 0x0cec1f76
Variable types: 8 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [2e+01, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+03]
Found heuristic solution: objective 356000.00000
Presolve removed 36 rows and 32 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 1 (of 1 available processors)

Solution count 1: 356000 

Optimal solution found (tolerance 1.00e-04)
Best 

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 32 primals, 0 duals
Objective: 3.56e+05
Solver model: available
Solver message: 2



('ok', 'optimal')

The result ('ok', optimal') indicates that the optimizer has found an optimal solution. Hence, you have checked that it is working. 

Now, you can for example show the status of every generator (if they are working or not)

In [16]:
network.generators_t.status

Generator-com,coal,gas
snapshot,,
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,0.0,1.0


Show the power produced by every generator 

In the final hour, load goes below part-load limit of coal generation (30%), forcing gas to commit.

In [17]:
network.generators_t.p

Generator,coal,gas
snapshot,,
0,4000.0,0.0
1,6000.0,0.0
2,5000.0,0.0
3,0.0,800.0


The total cost can be read from the network objetive.

In [18]:
network.objective

356000.0

You are now ready to start the MESM_project.